In [ ]:
# Cell 1: Setup logging with timestamps and durations
import logging
import os
import time
from datetime import datetime

class TimestampedLogger:
    def __init__(self, log_dir='logs', log_file='simulation.log'):
        os.makedirs(log_dir, exist_ok=True)
        self.log_path = os.path.join(log_dir, log_file)
        logging.basicConfig(filename=self.log_path, level=logging.INFO, filemode='w')
        self.start_time = time.time()
        self.last_time = self.start_time
        self.log("Logger initialized.")

    def _now(self):
        return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _duration(self):
        current_time = time.time()
        duration = current_time - self.last_time
        self.last_time = current_time
        return f"{duration:.3f}s"

    def log(self, message):
        timestamp = self._now()
        duration = self._duration()
        log_message = f"[{timestamp}] (+{duration}) {message}"
        print(log_message)
        logging.info(log_message)

logger = TimestampedLogger()


In [ ]:
# Cell 2: Define data structures for the simulation environment
from dataclasses import dataclass, field
from typing import Tuple, List, Optional
import random
import uuid

@dataclass
class Tile:
    x: int
    y: int
    piece: Optional[str] = None  # e.g. "King", "Pawn", etc.
    drones: List[str] = field(default_factory=list)  # e.g. ["Drone_1", "Drone_4"]

@dataclass
class Drone:
    id: int
    position: Tuple[int, int]
    memory: dict = field(default_factory=dict)

@dataclass
class King:
    """Represents the king on the board."""
    position: Tuple[int, int]

@dataclass
class SimulationState:
    """Tracks the state of the simulation."""
    grid_size: Tuple[int, int]
    tiles: List[List[Tile]]
    drones: List[Drone]
    king: King
    turn: int = 0


In [ ]:
# Cell 3: Initialize the simulation grid and entities
def initialize_simulation(grid_size: Tuple[int, int], num_drones: int) -> SimulationState:
    """Initializes the grid, drones, and king."""
    width, height = grid_size
    tiles = [[Tile(x=x, y=y) for y in range(height)] for x in range(width)]

    # Place king at the center
    king_pos = (width // 2, height // 2)
    king = King(position=king_pos)
    tiles[king_pos[0]][king_pos[1]].piece = 'King'

    # Place drones randomly, avoiding the King's tile
    drones = []
    occupied_positions = {king_pos}
    for i in range(num_drones):
        while True:
            pos = (random.randint(0, width - 1), random.randint(0, height - 1))
            if pos not in occupied_positions:
                break
        occupied_positions.add(pos)
        drone = Drone(
            id=i,
            position=pos
        )
        tiles[pos[0]][pos[1]].drones.append(f"Drone_{i}")
        drones.append(drone)

    return SimulationState(
        grid_size=grid_size,
        tiles=tiles,
        drones=drones,
        king=king,
        turn=0
    )


In [ ]:
# Cell 4: Load configuration from a JSON file
import json

def load_config(config_path: str = "config.json") -> dict:
    """Loads the configuration file."""
    with open(config_path, "r") as f:
        return json.load(f)


In [ ]:
# Cell 5: Define the simulation runner class
class Simulation:
    def __init__(self, config, state: SimulationState, logger: TimestampedLogger):
        self.config = config
        self.state = state
        self.logger = logger

    def run(self):
        self.logger.log("Starting simulation.")
        max_turns = self.config["simulation"].get("max_turns", 10)

        for turn in range(max_turns):
            self.state.turn = turn
            self.logger.log(f"Turn {turn + 1} started.")

            # Each drone could act here
            for drone in self.state.drones:
                self.logger.log(f"{drone.id} is at {drone.position}.")

            self.logger.log(f"Turn {turn + 1} ended.")
        self.logger.log("Simulation finished.")


In [ ]:
# Cell 6: Load config and initialize the simulation
config = load_config("config.json")
logger.log("Config loaded.")

state = initialize_simulation(
    grid_size=tuple(config["board"]["size"]),
    num_drones=config["simulation"]["num_drones"]
)
logger.log("Simulation initialized.")


In [ ]:
# Cell 7: Run the simulation
sim = Simulation(config, state, logger)
sim.run()


In [ ]:
# Cell 8: Setup Pygame and draw the initial field
import pygame
import sys

def draw_field(screen, state: SimulationState, tile_size=60):
    screen.fill((255, 255, 255))
    font = pygame.font.SysFont("Arial", 14)

    for x in range(state.grid_size[0]):
        for y in range(state.grid_size[1]):
            tile = state.tiles[x][y]
            rect = pygame.Rect(x * tile_size, y * tile_size, tile_size, tile_size)
            pygame.draw.rect(screen, (200, 200, 200), rect, 1)

            # Draw king
            if tile.piece == "King":
                pygame.draw.rect(screen, (255, 215, 0), rect)
                text = font.render("K", True, (0, 0, 0))
                screen.blit(text, (rect.x + 20, rect.y + 20))

    pygame.display.flip()

pygame.init()
grid_width, grid_height = state.grid_size
screen = pygame.display.set_mode((grid_width * 60, grid_height * 60))
pygame.display.set_caption("Drone Chessboard")
draw_field(screen, state)


In [ ]:
# Cell 9: Draw drones on the board (supporting multiple per tile)
def draw_drones(screen, state: SimulationState, tile_size=60):
    font = pygame.font.SysFont("Arial", 12)
    radius = 10

    # Group drones by tile
    tile_map = {}
    for drone in state.drones:
        key = tuple(drone.position)
        tile_map.setdefault(key, []).append(drone)

    for (x, y), drone_list in tile_map.items():
        base_x = x * tile_size + tile_size // 2
        base_y = y * tile_size + tile_size // 2

        # Stack drones vertically if there are multiple
        for idx, drone in enumerate(drone_list):
            offset = (idx - len(drone_list) / 2) * (radius * 2 + 2)
            drone_x = base_x
            drone_y = base_y + offset

            # Draw circle
            pygame.draw.circle(screen, (100, 149, 237), (int(drone_x), int(drone_y)), radius)

            # Draw drone ID number
            drone_label = font.render(str(drone.id)[-2:], True, (255, 255, 255))
            screen.blit(drone_label, (drone_x - radius + 2, drone_y - 6))

    pygame.display.flip()

# Call this after draw_field to update drone positions
draw_drones(screen, state)


In [ ]:
# Cell 10: Main simulation loop with live visual updates
def run_simulation_loop(state: SimulationState, logger: TimestampedLogger, steps: int = 10, delay: float = 0.5):
    logger.log(f"Starting simulation loop for {steps} steps.")
    for step in range(steps):
        logger.log(f"Simulation step {step + 1}")
        state.turn += 1

        # For now, we don't move drones; just update visuals
        draw_field(screen, state)
        draw_drones(screen, state)

        pygame.time.delay(int(delay * 1000))  # Delay in milliseconds

    logger.log("Simulation loop completed.")


In [ ]:
# Cell 11: Handle quit events to close Pygame window properly
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()
            logger.log("Pygame window closed.")
